# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [20]:
#import libraries
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#load data
df = pd.read_csv("homepage_actions.csv")
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [21]:
df[df["action"] == "click"]

,timestamp,id,group,action
8,2016-09-24 20:58:01.948663,349125,experiment,click
12,2016-09-24 21:06:27.553057,601714,experiment,click
15,2016-09-24 21:30:02.739756,487634,experiment,click
17,2016-09-24 23:01:12.108316,468601,experiment,click
21,2016-09-25 00:01:47.933853,555973,experiment,click
...,...,...,...,...
8162,2017-01-17 23:20:35.483601,451198,control,click
8164,2017-01-17 23:47:58.209653,252195,control,click
8167,2017-01-18 00:56:24.554729,344770,experiment,click
8180,2017-01-18 08:54:56.879682,615849,experiment,click


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [23]:
df.isnull().sum()

timestamp    0
id           0
group        0
action       0
dtype: int64

In [24]:
df["action"].value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [25]:
# number of those that viewed
nov = df[df["action"] == "view"]["id"].unique()
nov = set(nov)
len(nov)

6328

In [26]:
# number of those that clicked
noc = df[df["action"] == "click"]["id"].unique()
noc = set(noc)
len(noc)

1860

In [27]:
# Those who viewed but did not click
len(nov - noc)

4468

In [28]:
# Those who clicked but did not view
len(noc - nov)

0

All those who clicked the homepage also viewed it.

In [29]:
#control group
control = df[df["group"] == "control"]
control.head()

,timestamp,id,group,action
3,2016-09-24 19:59:02.646620,671993,control,view
9,2016-09-24 21:00:12.278374,560027,control,view
23,2016-09-25 00:25:14.141290,281985,control,view
24,2016-09-25 01:14:48.603202,407864,control,view
25,2016-09-25 02:16:11.046654,342984,control,view


In [30]:
#experiment group
experiment = df[df["group"] == "experiment"]
experiment.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
5,2016-09-24 20:32:25.712659,681598,experiment,view


In [31]:
#check for overlap of groups
c_ids = control["id"].unique()
c_ids = set(c_ids)
e_ids = experiment["id"].unique()
e_ids = set(e_ids)
c_ids & e_ids

set()

There is no overlap between the two groups

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [34]:
df['count'] = 1

control = df[df.group=="control"].pivot(index = "id", columns = "action", values = "count")
control = control.fillna(value=0)

experiment = df[df.group == "experiment"].pivot(index = "id", columns = "action", values = "count")
experiment = experiment.fillna(value=0)

import scipy.stats as stats
results = stats.ttest_ind(control.click, experiment.click, equal_var = False)
results

Ttest_indResult(statistic=-2.615440020788211, pvalue=0.008932805628674201)

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [35]:
#Your code here
control_mean = control.click.mean()
expected_clicks = control_mean * len(experiment)
expected_clicks

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [37]:
#Your code here
import numpy as np
n = len(experiment)
p = control_mean
var = n * p * (1 - p)
std = np.sqrt(var)
std

24.568547907005815

In [38]:
z = (experiment.click.sum() - expected_clicks) / std
z

3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [40]:
#Your code here
p = 1 - stats.norm.cdf(z)
p

0.00012486528006949715

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

The z-test pvalue is slightly lower but they are both less than than an alpha of 0.05. This shows that the experiment homepage is more effective than the existing homepage.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.